<a href="https://colab.research.google.com/github/seoyeon7/ML/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [377]:
from sklearn import datasets
iris = datasets.load_iris()
samples = iris.data

In [378]:
import pandas as pd
data = pd.DataFrame(samples, columns=feature_names)
data['target'] = iris['target']
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [379]:
data = data.sample(frac=1).reset_index(drop=True)
data['seq'] = data.index
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,seq
0,5.5,2.6,4.4,1.2,1,0
1,5.1,3.3,1.7,0.5,0,1
2,6.5,3.0,5.2,2.0,2,2
3,5.0,3.3,1.4,0.2,0,3
4,5.4,3.4,1.7,0.2,0,4


In [380]:
#train set(80%)과 test set(20%)으로 분류
train_size = int(data.shape[0]*0.80)
test_size = int(data.shape[0]*0.20)

train = data[:train_size]
test = data[test_size:]

In [381]:
# 거리 계산 함수

In [382]:
# Euclidian distance (유클리드 거리)
import math

def get_eucl(row1, row2):
    return math.sqrt(sum([(x1-x2)**2 for x1,x2 in zip(row1,row2)]))

In [383]:
# Cosine Similarity (코사인 유사도)
def get_cos_sim(row1, row2):
    return math.acos(
        sum([x1*x2 for x1,x2 in zip(row1,row2)])/(sum([i**2 for i in row1]) * sum([i**2 for i in row2]))
    )

In [384]:
# Normalized Euclidean distance (정규화된 데이터에 대한 유클리드 거리를 계산)

train_n = pd.DataFrame(columns=['sepal length (cm)', 'sepal width (cm)','petal length (cm)','petal width (cm)', 'target', 'seq'])

train_n['target'] = train['target'].copy()
train_n['seq'] = train['seq'].copy()

train_n['sepal length (cm)'] = train['sepal length (cm)'].apply(
    lambda x: (x-train['sepal length (cm)'].min())/(train['sepal length (cm)'].max()-train['sepal length (cm)'].min()))

train_n['sepal width (cm)'] = train['sepal width (cm)'].apply(
    lambda x: (x-train['sepal width (cm)'].min())/(train['sepal width (cm)'].max()-train['sepal width (cm)'].min()))

train_n['petal length (cm)'] = train['petal length (cm)'].apply(
    lambda x: (x-train['petal length (cm)'].min())/(train['petal length (cm)'].max()-train['petal length (cm)'].min()))

train_n['petal width (cm)'] = train['petal width (cm)'].apply(
    lambda x: (x-train['petal width (cm)'].min())/(train['petal width (cm)'].max()-train['petal width (cm)'].min())) 


In [385]:
# 거리 계산

In [386]:
train_val = train.values
ntrain_val = train_n.values
eud = []
cosine_sim = []
neud = []
l = len(train_val)
for i in range(l):
    eu_distance = []
    cos_sim = []
    neu_distance = []
    for j in range(l):
        if(i!=j):
            index = train_val[j][5]
            nindex = ntrain_val[j][5]
            ed = get_eucl(train_val[i][:-2], train_val[j][:-2])
            cs = get_cos_sim(train_val[i][:-2], train_val[j][:-2])
            neu = get_eucl(ntrain_val[i][:-2], ntrain_val[j][:-2])
            
            eu_distance.append((ed, index))
            cos_sim.append((cs, index))
            neu_distance.append((neu, nindex))
            
    eu_distance.sort(key= lambda x: x[0])
    cos_sim.sort(key= lambda x: x[0])
    neu_distance.sort(key= lambda x: x[0])
    
    eu_distance = [i[1] for i in eu_distance]
    cos_sim = [i[1] for i in cos_sim]
    neu_distance = [i[1] for i in neu_distance]
    
    eud.append(eu_distance)
    cosine_sim.append(cos_sim)
    neud.append(neu_distance)
    

train['euclidean'] = eud
train['cosine_sim'] = cosine_sim
train['n_euclidean'] = neud

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [387]:
# 가까운 요소 k개 선택

def get_near(row, dis,k):
    return row[dis][:k]

In [388]:
# k개 요소 중 dominant class 선택

def get_dominant_class(df, neighbors):
    classes = df[df['seq'].isin(neighbors)]['target']
    return classes.value_counts().index[0]

In [389]:
# 가장 가까운 k개의 요소 선택 (k=3으로 설정)

test['seq'] = test.index
test2 = test.values
test_eud = []
l = len(test)
for i in range(l):
    test_eu_dist = []
    for j in range(len(train)):
        index = train_val[j][5]
        ed = get_eucl(test2[i][:-2], train_val[j][:-2])
        test_eu_dist.append((ed, index))
        
    test_eu_dist.sort(key= lambda x: x[0])
    test_eu_dist = [i[1] for i in test_eu_dist]
    test_eud.append(test_eu_dist)
    
test['euclidean'] = test_eud

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [390]:
# 가장 가까운 k개의 요소 선택 (k=3으로 설정)
test['eu'] = test.apply(lambda x: get_near(x, 'euclidean',3), axis=1)
test[feature_names+['eu']].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),eu
30,5.8,2.8,5.1,2.4,"[30.0, 91.0, 6.0]"
31,6.6,3.0,4.4,1.4,"[31.0, 19.0, 16.0]"
32,6.3,3.3,6.0,2.5,"[32.0, 57.0, 95.0]"
33,5.4,3.9,1.3,0.4,"[33.0, 56.0, 110.0]"
34,6.7,3.3,5.7,2.1,"[34.0, 66.0, 75.0]"


In [391]:
# 3개의 k와 가장 가까운 dominant class 선택
test['eu_class'] = test['eu'].apply(lambda row: get_dominant_class(train, row))
test[feature_names+['eu_class']].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),eu_class
30,5.8,2.8,5.1,2.4,2
31,6.6,3.0,4.4,1.4,1
32,6.3,3.3,6.0,2.5,2
33,5.4,3.9,1.3,0.4,0
34,6.7,3.3,5.7,2.1,2


In [392]:
test_acc = test[test['target']==test['eu_class']].shape[0]/test.shape[0]
print('Test Accuracy: {:2.4f}%'.format(test_acc*100))

Test Accuracy: 96.6667%
